In [1]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ["OPENAI_API_KEY"]

In [2]:
%pip install pypdf 

In [241]:
from langchain.document_loaders import PyPDFLoader
loaders =[ 
    PyPDFLoader("D:\Documents\CDC Internship\ChatBot\Contact Us.pdf"),
    PyPDFLoader("D:\Documents\CDC Internship\ChatBot\Overview.pdf"),
    PyPDFLoader("D:\Documents\CDC Internship\ChatBot\CDC FAQs.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [247]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

In [248]:
splits = text_splitter.split_documents(docs)

In [249]:
len(splits)

47

In [250]:
splits[39].page_content[:350]

'All the Notices and Circulars issued by CDC are available in the respective heads on the Home page of the CDC Access portal w hich can be \nviewed without logging into the portal.  \n \nFrom where I can see the details of upcoming IPOs on Web Access portal?  \n \nYou c an check the details of upcoming IPOs from the Upcoming IPO tab available on the Home'

In [181]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [182]:
from langchain.vectorstores import Chroma

In [14]:
%pip install tiktoken

  Using cached tiktoken-0.4.0.tar.gz (25 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for tiktoken: filename=tiktoken-0.4.0-cp38-cp38-win32.whl size=673801 sha256=f3551d92f152c5b65f29eddafa6aae02424b947377412c913af2b985e719f97c
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\96\ff\3b\c5afbc8d5c24c430cb15aa5a133a4618685e1f83292f2ae7be
Successfully built tiktoken



  Using cached tiktoken-0.4.0.tar.gz (25 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for tiktoken: filename=tiktoken-0.4.0-cp38-cp38-win32.whl size=673799 sha256=cee163fecda52c7b17c7c53c0d3cd850038ae14eb689ad7a9ff811f21a44406b
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\96\ff\3b\c5afbc8d5c24c430cb15aa5a133a4618685e1f83292f2ae7be
Successfully built tiktoken


In [251]:
persist_directory= 'D:\Documents\chroma'
!rm -rf ./Documents/chroma
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)
print(vectordb._collection.count()) 



'rm' is not recognized as an internal or external command,
operable program or batch file.


47


In [25]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo-0301


In [28]:
question = "What does CDC offer?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [252]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

Retrieval Question/Answer Chain

In [30]:
from langchain.chains import RetrievalQA

In [31]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [32]:
result = qa_chain({"query": question})

In [33]:
result["result"]

'CDC offers a customer convenience initiative in the form of a web portal that provides free-of-cost facilities. These facilities include CDC Share Registrar Services Limited, which allows retail and corporate investors to open and maintain custody accounts directly with CDC, and Trustee and Custodial Services for customer dealing on behalf of companies. CDC also has a wholly-owned subsidiary called ITMinds Limited.'

Prompt Generation


In [347]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """You are a customer service chatbot for the Central Depository Company of Pakistan. Use the following pieces of context to answer the question at the end. If and only if the user asks in roman urdu then try to answer the question according to the context in roman urdu, don't use hindi words in your answer. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use simple words and three sentences maximum. Keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [352]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [361]:
question = 'Do i qualify to make a cdc access account'

In [362]:
result = qa_chain({"query": question})

In [363]:
result["result"]

"Anyone, including individuals, corporations, residents, non-residents, and foreigners, can register for a CDC Access account, whether they are new to investing or already have physical shares or a CDS account. You can sign up for a new account even if you don't have a CDS account or physical securities."